In [1]:
# code is adapted from CADDA and braincode
from audioop import reverse
from enum import auto
from numbers import Real
from operator import invert
import random
import numpy as np
import torch
import torch.nn as nn
from scipy.interpolate import Rbf
from scipy.spatial.transform import Rotation
from sklearn.utils import check_random_state
from torch.fft import fft, ifft
from torch.nn.functional import dropout2d, pad, one_hot
from torch.distributions import Normal
from mne.filter import notch_filter
from mne.channels.interpolation import _make_interpolation_matrix
from mne.channels import make_standard_montage
import matplotlib.pyplot as plt
from ecgdetectors import Detectors
from scipy.interpolate import CubicSpline

target_dir = 'subdiag'
num_class = 23
n_fold = 10
predict_mode = 'test'

d:\anaconda\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### torch adaptive test
bs,ch,seqlen = 1,1,100
signal = torch.randn(bs,seqlen,ch)
#print(signal)
windowed_sig = torch.nn.functional.avg_pool1d(signal.view(1,1,seqlen),kernel_size=10, stride=1, padding=5,count_include_pad=False).view(1,-1)[:,:seqlen]
print(windowed_sig.shape,windowed_sig)
quant_score = torch.quantile(windowed_sig[0],0.8)
print('quant: ',quant_score)
#keep adapt ways
select_windows = (windowed_sig[0] >= quant_score).nonzero(as_tuple=True)[0].detach().cpu().numpy()
#select_windows = select_windows + 10 // 2
#keep auto ways
passed_windows = []
for x in range(100):
    x1 = np.clip(x - 10 // 2, 0, seqlen)
    x2 = np.clip(x + 10 // 2, 0, seqlen)
    reg_mean = signal[0,x1: x2].mean()
    if reg_mean >= quant_score:
        passed_windows.append(reg_mean)
passed_windows = np.array(passed_windows)
print('Compare two result')
print('Adapt ',windowed_sig[0,select_windows])
print('Auto ',passed_windows)

torch.Size([1, 100]) tensor([[ 0.1610,  0.1462,  0.0132,  0.1305,  0.0163,  0.0271, -0.0641, -0.1368,
         -0.1287, -0.0022,  0.0184, -0.0795, -0.0469, -0.0536,  0.0172, -0.0254,
          0.0190, -0.0621,  0.1919,  0.3247,  0.1226,  0.2166,  0.3186,  0.2340,
          0.2878,  0.4018,  0.2780,  0.2582, -0.0120, -0.0971,  0.0680,  0.0574,
         -0.0766, -0.0753, -0.0578,  0.0140,  0.0276,  0.1958,  0.1602,  0.2277,
          0.1999,  0.3660,  0.4377,  0.4548,  0.5833,  0.4884,  0.5764,  0.4850,
          0.5466,  0.4853,  0.4696,  0.2693,  0.2994,  0.2527,  0.2489,  0.2137,
          0.0889,  0.0355,  0.1358,  0.2004,  0.1693,  0.2082,  0.1607,  0.1911,
         -0.1076, -0.0616,  0.0681,  0.0660,  0.1145, -0.1072, -0.1581, -0.1767,
         -0.1959, -0.2209, -0.0905, -0.2346, -0.2838, -0.2980, -0.3967, -0.3930,
         -0.6026, -0.5070, -0.4560, -0.3028, -0.4097, -0.3242, -0.2523, -0.0897,
         -0.0265,  0.0647,  0.3243,  0.0934,  0.0644, -0.0732, -0.0803, -0.0532,
       

In [7]:
from sklearn.model_selection import StratifiedShuffleSplit,ShuffleSplit
dummy = np.array([0,1,2,3,3,6,4,6,6,6,6,6,6,2,2,1,3,5,5,5])
unique, counts = np.unique(dummy, return_counts=True)
counts_array = np.asarray((unique, counts)).T
print(counts_array)
too_minor_class = unique[counts < 2] #11/09 change for minor class
total = len(dummy)
rd_idxs,too_minor_sample = [],[]
dummy_reduced = []
for i in range(total):
    if dummy[i] in too_minor_class:
        too_minor_sample.append(i)
    else:
        rd_idxs.append(i)
        dummy_reduced.append(dummy[i])

print(rd_idxs)
print(too_minor_sample)
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
sss = sss.split(list(rd_idxs), dummy_reduced)
tot_train_idx, search_idx = next(sss)
too_minor_sample = np.array(too_minor_sample)
tot_train_idx = np.concatenate((tot_train_idx,too_minor_sample)) #add back minor sample, 11/09
print(tot_train_idx)
print(search_idx)
print(f'Train len: {len(tot_train_idx)},Search len: {len(search_idx)}')

[[0 1]
 [1 2]
 [2 3]
 [3 3]
 [4 1]
 [5 3]
 [6 7]]
[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[0, 6]
[ 7 14 17 16  1  0  8  6  5  0  6]
[11  9  4 15 13  2  3 12 10]
Train len: 11,Search len: 9
